In [1]:
from NGram.NGram import NGram
from SpellChecker.SimpleSpellChecker import SimpleSpellChecker
from MorphologicalAnalysis.FsmMorphologicalAnalyzer import FsmMorphologicalAnalyzer
import xml
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtPrintSupport import *
import duzelt
import os
import sys

class MainWindow(QMainWindow):

    def __init__(self, *args, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)
        layout = QVBoxLayout()
        self.editor = QPlainTextEdit()   
        fixedfont = QFontDatabase.systemFont(QFontDatabase.FixedFont)
        fixedfont.setPointSize(12)
        self.editor.setFont(fixedfont)
        self.path = None
        layout.addWidget(self.editor)
        self.listwidget = QListWidget ()
        self.listwidget.setFont (fixedfont)
        layout.addWidget(self.listwidget)
        self.listwidget.hide()
        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)
        self.status = QStatusBar()
        self.setStatusBar(self.status)
        file_toolbar = QToolBar("Dosya")
        file_toolbar.setIconSize(QSize(32, 32))
        self.addToolBar(file_toolbar)
        file_menu = self.menuBar().addMenu("&Dosya")
        open_file_action = QAction(QIcon(os.path.join('image', 'openfile.png')), "Dosya Aç", self)
        open_file_action.setStatusTip("Dosya Aç")
        open_file_action.triggered.connect(self.file_open)
        file_menu.addAction(open_file_action)
        file_toolbar.addAction(open_file_action)
        save_file_action = QAction(QIcon(os.path.join('image', 'save.png')), "Kaydet", self)
        save_file_action.setStatusTip("Kaydet")
        save_file_action.triggered.connect(self.file_save)
        file_menu.addAction(save_file_action)
        file_toolbar.addAction(save_file_action)
        saveas_file_action = QAction(QIcon(os.path.join('image', 'saveas.png')), "Farklı Kaydet", self)
        saveas_file_action.setStatusTip("Farklı Kaydet")
        saveas_file_action.triggered.connect(self.file_saveas)
        file_menu.addAction(saveas_file_action)
        file_toolbar.addAction(saveas_file_action)
        edit_toolbar = QToolBar("Düzenle")
        edit_toolbar.setIconSize(QSize(32, 32))
        self.addToolBar(edit_toolbar)
        edit_menu = self.menuBar().addMenu("&Düzenle")
        undo_action = QAction(QIcon(os.path.join('image', 'gerial.png')), "Geri Al", self)
        undo_action.setStatusTip("Değişikliği Geri Al")
        undo_action.triggered.connect(self.editor.undo)
        edit_toolbar.addAction (undo_action)
        edit_menu.addAction(undo_action)
        redo_action = QAction(QIcon(os.path.join('image', 'yinele.png')), "Yinele", self)
        redo_action.setStatusTip("Değişikliği Yinele")
        redo_action.triggered.connect(self.editor.redo)
        edit_toolbar.addAction(redo_action)
        edit_menu.addAction(redo_action)
        edit_menu.addSeparator()
        cut_action = QAction(QIcon(os.path.join('image', 'cut.png')), "Kes", self)
        cut_action.setStatusTip("Kes")
        cut_action.triggered.connect(self.editor.cut)
        edit_toolbar.addAction(cut_action)
        edit_menu.addAction(cut_action)
        copy_action = QAction(QIcon(os.path.join('image', 'copy.png')), "Kopyala", self)
        copy_action.setStatusTip("Kopyala")
        copy_action.triggered.connect(self.editor.copy)
        edit_toolbar.addAction(copy_action)
        edit_menu.addAction(copy_action)
        paste_action = QAction(QIcon(os.path.join('image', 'paste.png')), "Yapıştır", self)
        paste_action.setStatusTip("Yapıştır")
        paste_action.triggered.connect(self.editor.paste)
        edit_toolbar.addAction(paste_action)
        edit_menu.addAction(paste_action)
        select_action = QAction(QIcon(os.path.join('image', 'selectall.png')), "Tümünü Seç", self)
        select_action.setStatusTip("Tümünü Seç")
        select_action.triggered.connect(self.editor.selectAll)
        edit_menu.addAction(select_action)
        edit_menu.addSeparator()
        wrap_action = QAction(QIcon(os.path.join('image', 'sıgdır.png')), "Metni Pencereye Sığdır", self)
        wrap_action.setStatusTip("Metni Pencereye Sığdır")
        wrap_action.setCheckable(True)
        wrap_action.setChecked(True)
        wrap_action.triggered.connect(self.edit_toggle_wrap)
        edit_menu.addAction(wrap_action)
        Duzelt_toolbar = QToolBar("Düzelt")
        Duzelt_toolbar.setIconSize(QSize(32, 32))
        self.addToolBar(Duzelt_toolbar)
        Duzelt_menu = self.menuBar().addMenu("&Düzelt") 
        checkspell_action = QAction(QIcon(os.path.join('image', 'duzelt.png')), "Yazım Denetimi", self)
        checkspell_action.setStatusTip("Yazım Denetimi Yap")
        checkspell_action.triggered.connect(self.checkSpell)
        Duzelt_toolbar.addAction(checkspell_action)
        Duzelt_menu.addAction(checkspell_action)
        edit_menu.addSeparator()
        info_action = QAction(QIcon(os.path.join('image', 'info.png')), "Bilgi Penceresini Aç/Kapat", self)
        info_action.setStatusTip("Bilgi Penceresini Açar veya Kapatır")
        info_action.triggered.connect(self.openClose)
        Duzelt_toolbar.addAction(info_action)
        Duzelt_menu.addAction(info_action)
        self.duzelt_ = duzelt.Duzelt ()
        self.update_title()
        self.show()
        
    def dialog_critical(self, s):
        dlg = QMessageBox(self)
        dlg.setText(s)
        dlg.setIcon(QMessageBox.Critical)
        dlg.show()
    
    def update_title(self):
        self.setWindowTitle("%s Düzgün" % (os.path.basename(self.path) if self.path else "Doğru")) 

    def edit_toggle_wrap(self):
        self.editor.setLineWrapMode( 1 if self.editor.lineWrapMode() == 0 else 0 )
        
    def openClose(self):
        if self.listwidget.isVisible()==1:
            self.listwidget.hide()
        else:
            self.listwidget.show()

    def file_open(self):
        path, _ = QFileDialog.getOpenFileName(self, "Dosya Aç", "", "Metin Dosyaları (*.txt)")
        if path:
            try:
                with open(path, 'r',encoding="UTF-8") as f:
                    text = f.read()
            except Exception as e:
                self.dialog_critical(str(e))
            else:
                self.path = path
                self.editor.setPlainText(text)
                self.update_title()
                  
    def file_save(self):
        if self.path is None:
            return self.file_saveas()
        self._save_to_path(self.path)
 
    def file_saveas(self):
        path, _ = QFileDialog.getSaveFileName(self, "Dosya Kaydet", "", "Metin Dosyaları (*.txt)")
        if not path:
            return
        self._save_to_path(path) 
        
    def _save_to_path(self, path):
        text = self.editor.toPlainText()
        try:
            with open(path, 'w',encoding="UTF-8") as f:
                f.write(text)
        except Exception as e:
            self.dialog_critical(str(e))
        else:
            self.path = path
            self.update_title()
            
    def cleanhtml(raw_html):
        return
                
    def checkSpell(self):
        self.listwidget.clear ()
        self.editor.setLineWrapMode(0)
        textboxValue = self.editor.toPlainText ().rstrip()
        textboxValue = textboxValue.replace("\n", " *** ")
        ws= textboxValue.split(" ")
        self.editor.clear()
        suggestions =""
        control = self.duzelt_.checkSpell(textboxValue.rstrip())
        print(control)        
        newText =""
        for count, w in enumerate(ws):
            print(w)
            if w=="***":
                newText += "<br>"
            elif (control[count]==0):
                newText += w+" "
            elif (control[count]==1):
                newText += '<span style="background-color:  #abe670";color:white>'+str(w)+'</span>' + " "
            elif (control[count]==2):
                if not (self.duzelt_.isCorrect (str(w))):
                    newText += '<span style="background-color: #deca66";color:white>'+str(w)+'</span>' + " "      
            elif (control[count] == 3):
                newText += '<span style="background-color: #e3acb4";color:yellow>'+str(w)+'</span>' + " "   
            elif (control[count] == 4):
                newText += '<span style="background-color: #cb95f5";color:green>'+str(w)+'</span>' + " "       
            elif (control[count] == 5):
                newText += '<span style="background-color: #3dbcd9";color:white>'+str(w)+'</span>' + " "       
            elif (control[count] == 6):
                newText += '<span style="background-color: #d90d69";color:brown>'+str(w)+'</span>' + " "       
        print(newText)
        self.editor.appendHtml(newText)
        test = 1
        self.listwidget.clear()
        self.listwidget.show()
        colors = ['#deca66', '#abe670', '#e3acb4', '#cb95f5', '#d90d69','#3dbcd9']
        false = QListWidgetItem(' Yazım yanlışı bulunmaktadır. ')
        false.setBackground( QColor(colors[0]) )
        self.listwidget.addItem(false)
        false = QListWidgetItem(' Noktalama işaretinden önce boşluk bırakılmamalıdır. ')
        false.setBackground( QColor(colors[1]) )
        self.listwidget.addItem(false)
        false = QListWidgetItem(' Cümlenin sonuna nokta koyulmalıdır.')
        false.setBackground( QColor(colors[2]) )
        self.listwidget.addItem(false)
        false = QListWidgetItem(' Noktalama işaretinden sonra boşluk bırakılmalıdır. ')
        false.setBackground( QColor(colors[3]) )
        self.listwidget.addItem(false)
        false = QListWidgetItem(' Türkçe kelime önerisi bulunmaktadır. ')
        false.setBackground( QColor(colors[4]) )
        self.listwidget.addItem(false)
        false = QListWidgetItem(' Cümleye büyük harfle başlanmalıdır. ')
        false.setBackground( QColor(colors[5]) )
        self.listwidget.addItem(false)
        self.editor.setLineWrapMode (1)  
        
    def mousePressEvent(self, event):
        try:
            if event.button () == Qt.RightButton:
                print ('right') 
                textCursor = self.editor.cursorForPosition (QPoint(event.pos().x(), event.pos().y()-70))
                textCursor.select (QTextCursor.WordUnderCursor)
                self.editor.setTextCursor (textCursor)
                self.word = textCursor.selectedText ()
                print("word : ",self.word)
            menu = QMenu ()
            VelAction = menu.addAction ("Düzelt")
            menu.addSeparator ()
            if self.word != "" or len (self.word) > 1 or self.word != " ":
                print ("burada")
                print (self.duzelt_.isCorrect (self.word))
                punctuation = [".", ",", "?", "!", "...", ":", "(", ")"]
                if self.word in punctuation:
                    w = menu.addAction ("Noktalama İşaretinden Önceki Boşluk Kaldırılmalıdır.")
                if not (self.duzelt_.isCorrect (self.word)):
                    for i, w in enumerate (set (self.duzelt_.wordSuggestion (self.word))):
                        w = menu.addAction (w)
                elif len(self.duzelt_.turkishWordSuggestion(self.word))>0:
                    for w in self.duzelt_.turkishWordSuggestion(self.word):
                        w = menu.addAction(w)
            action = menu.exec_ (self.mapToGlobal (event.pos ()))
            print(str(action.text()))
            menu.addSeparator ()
            if action != VelAction:
                self.change(str(action.text()))
            menu.addSeparator ()
            if event.button()==Qt.LeftButton:
                pass
        except:
            pass

    def change(self,w):
        txt = self.editor.toPlainText ().rstrip()
        print(txt)
        cursor = self.editor.textCursor ()
        if (w =="Noktalama İşaretinden Önceki Boşluk Kaldırılmalıdır."):
            txt = txt[:cursor.selectionStart ()]  + self.word + " " + txt[cursor.selectionEnd ():]
        else:
            txt = txt[:cursor.selectionStart ()] +" "+ w+" " + txt[ cursor.selectionEnd ():]
        self.editor.clear ()
        txt = txt.replace ("\n", " <br> ")
        print(txt)
        self.editor.appendHtml((txt))
        self.checkSpell()
        
if __name__ == '__main__':

    app = QApplication(sys.argv)
    app.setApplicationName("Düzelt")

    window = MainWindow()
    app.exec_()


[3]
selm
<span style="background-color: #e3acb4";color:yellow>selm</span> 


UnboundLocalError: local variable 'variable' referenced before assignment

In [1]:
import sys
sys.path
sys.path.append('C:\\Users\\pc\\AppData\\Local\\Programs\\Python\\Python39\\Lib\\site-packages')
sys.path
sys.path.append('C:\\Users\\pc\\anaconda3\\lib\\site-packages')
sys.path

['C:\\Users\\pc\\Desktop\\bitirme\\düzeltme\\velhasil-Velhasil',
 '',
 'C:\\Users\\pc\\anaconda3\\lib\\site-packages',
 'C:\\Users\\pc\\AppData\\Local\\Programs\\Python\\Python39\\Lib\\site-packages',
 'C:\\Users\\pc\\Desktop\\bitirme\\düzeltme\\velhasil-Velhasil',
 'C:\\Users\\pc\\anaconda3\\python38.zip',
 'C:\\Users\\pc\\anaconda3\\DLLs',
 'C:\\Users\\pc\\anaconda3\\lib',
 'C:\\Users\\pc\\anaconda3',
 'C:\\Users\\pc\\anaconda3\\lib\\site-packages\\PyDispatcher-2.0.5-py3.5.egg',
 'C:\\Users\\pc\\anaconda3\\lib\\site-packages\\locket-0.2.1-py3.8.egg',
 'C:\\Users\\pc\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\pc\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\pc\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\pc\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\pc\\.ipython',
 'C:\\Users\\pc\\AppData\\Local\\Programs\\Python\\Python39\\Lib\\site-packages',
 'C:\\Users\\pc\\anaconda3\\lib\\site-packages']